In [ ]:
# remove this
import sys
#sys.path.append("/Users/harrison37/Work/alpine/ascent/install-debug/python-modules/")
sys.path.append("/Users/larsen30/research/test_builds/release/ascent/install/python-modules")

In [ ]:
# cleanup any old results
!./cleanup.sh

In [ ]:
# Ascent imports
# ascent + conduit imports
import conduit
import conduit.blueprint
import ascent

import numpy as np

# Jupyter imports
from IPython.display import Image
# helper we use when displaying results in the notebook
img_display_width = 500

In [ ]:
# create example mesh using conduit blueprint
mesh = conduit.Node()
conduit.blueprint.mesh.examples.braid("hexs",
                                      25,
                                      25,
                                      25,
                                      mesh)


In [ ]:
# make sure the mesh we created conforms to the blueprint
verify_info = conduit.Node()
if not conduit.blueprint.mesh.verify(mesh,verify_info):
    print("Mesh Verify failed!")
    print(verify_info.to_yaml())
else:
    print("Mesh verify success!")

## Relay Extract Example
### Save input mesh and all fields to Blueprint HDF5 Files

In [ ]:
# Use Ascent to Export the data
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
# add a relay extract that will write blueprint hdf5 files
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_extracts"
extracts = add_act["extracts"]
extracts["e1/type"] = "relay"
extracts["e1/params/path"] = "out_export_braid_all_fields"
extracts["e1/params/protocol"] = "blueprint/mesh/hdf5"

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions);

# close ascent
a.close()

In [ ]:
# Look at the Blueprint HDF5 extract with VisIt
# !visit -o out_export_braid_all_fields.cycle_000100.root

## Relay Extract Example
### Save input mesh and one selected field to Blueprint HDF5 Files

In [ ]:
# Use Ascent to Export the data
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_extracts"
extracts = add_act["extracts"]
extracts["e1/type"] = "relay"
extracts["e1/params/path"] = "out_export_braid_one_field"
extracts["e1/params/protocol"] = "blueprint/mesh/hdf5"

#
# add fields paramter to limit the export to the braid field
#
extracts["e1/params/fields"].append().set("braid")

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions);

# close ascent
a.close()

In [ ]:
# Look at the Blueprint HDF5 extract with VisIt
# !visit -o out_export_braid_one_field.cycle_000100.root

### Relay Extract Example
### Save pipeline contour results to Blueprint HDF5 Files

In [ ]:
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

pipelines = conduit.Node()

pipelines["pl1/f1/type"] = "contour"
contour_params = conduit.Node()
contour_params["field"] = "braid"

iso_vals = np.array([0.2, 0.4], dtype=np.float64)
contour_params["iso_values"].set_external(iso_vals)
pipelines["pl1/f1/params"] = contour_params

# setup actions
actions = conduit.Node()
add_act = actions.append();
add_act["action"] = "add_pipelines";
add_act["pipelines"] = pipelines;

extracts = conduit.Node()
extracts["e1/type"] = "relay"
extracts["e1/pipeline"]  = "pl1"
extracts["e1/params/path"] = "out_extract_braid_contour"
extracts["e1/params/protocol"] = "blueprint/mesh/hdf5"

# setup actions
add_act2 = actions.append()
add_act2["action"] = "add_extracts"
add_act2["extracts"] = extracts

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions);

# close ascent
a.close()

In [ ]:
# Look at the Blueprint HDF5 extract with VisIt
# !visit -o out_extract_braid_contour.cycle_000100.root

## Cinema Extract Example
### Create a Cinema Image Database with several renders of the contour results

In [ ]:
# Use Ascent to Export the data
a = ascent.Ascent()
a.open()

# publish mesh to ascent
a.publish(mesh);

pipelines = conduit.Node()
pipelines["pl1/f1/type"] = "contour"

contour_params = conduit.Node()
contour_params["field"] = "braid"
iso_vals = np.array([0.2, 0.4],dtype=np.float32)
contour_params["iso_values"].set_external(iso_vals)
pipelines["pl1/f1/params"] = contour_params

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_pipelines"
add_act["pipelines"] = pipelines

# declare a scene to render the dataset
scenes = conduit.Node()
scenes["s1/plots/p1/type"] = "pseudocolor";
scenes["s1/plots/p1/pipeline"] = "pl1";
scenes["s1/plots/p1/field"] = "braid";
# cinema params
scenes["s1/renders/r1/type"] = "cinema"
# use 5 renders in phi
scenes["s1/renders/r1/phi"] = 5
# and 5 renders in theta
scenes["s1/renders/r1/theta"] = 5
scenes["s1/renders/r1/db_name"] = "out_extract_cinema_contour"

add_act2 = actions.append()
add_act2["action"] = "add_scenes"
add_act2["scenes"] = scenes

# print our full actions tree
print(actions.to_yaml())

# execute
a.execute(actions);

# close ascent
a.close()

In [ ]:
from IPython.display import HTML
HTML(filename="cinema_databases/out_extract_cinema/index.html")